In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [33]:
Localidades = pd.read_csv("..\Datasets\Localidades.csv",delimiter = ',',encoding = "utf-8")
Clientes = pd.read_csv("..\Datasets\Clientes.csv",encoding = "utf-8",delimiter = ';', usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
Proveedores = pd.read_csv("..\Datasets\Proveedores.csv",delimiter = ',',encoding = "ansi")
Gasto = pd.read_csv("..\Datasets\Gasto.csv",delimiter = ',',encoding = "utf-8")
Compra = pd.read_csv("..\Datasets\Compra.csv",delimiter = ',',encoding = "utf-8")
Sucursales = pd.read_csv("..\Datasets\Sucursales.csv",delimiter = ';',encoding = "utf-8")
Venta = pd.read_csv("..\Datasets\Venta.csv",delimiter = ',',encoding = "utf-8")

In [34]:
#Creo dataframe con nombres e id de Provincias
Provincia = pd.DataFrame()

Tabla Venta

In [35]:
def CargaVentas():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Vent*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    Venta=pd.concat(df_list)
    #TABLA Venta
    Q1 = Venta["Precio"].quantile(0.25)
    Q3 = Venta["Precio"].quantile(0.75)
    #print(Q1)
    #print(Q3)
    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR))
    #print(outliersSup)

    #Uso la mascara para filtrar
    mask = Venta["Precio"]<outliersSup
    VentaSinOut2 = Venta[mask]
    #VentaSinOut2.describe()

    #Calculo la cantidad de outliers
    #Venta.shape[0]-VentaSinOut2.shape[0]

    #Outliers de precio
    Precio_Producto= VentaSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})

    Venta = pd.merge(Venta,Precio_Producto, how="left", on=["IdProducto"])
    Venta.sort_values("IdVenta")

    #Se reemplazan los 0 de precios2 por el valor de la media de los precios sin outlier
    Venta.Precio2.fillna(VentaSinOut2.Precio.mean(), inplace=True)

    #Se reemplazan con 0 y 1 todos los valores de precios vacios y cantidades
    Venta.Precio.fillna(0, inplace= True)
    Venta.Cantidad.fillna(1, inplace= True)

    #Hago columnas de unos
    Venta['Outliers'] = np.ones(Venta.shape[0])

    #Marco con 0 los outliers y despues los reemplazo con los valores de precio2

    Venta.loc[(Venta['Precio']>=outliersSup),'Outliers'] = 0
    #Reemplazo 0 y los outliers por el precio promedio sin outliers (precio2)

    for i in range(0,Venta.shape[0]):
        if Venta.Precio[i] == 0 or Venta.Precio[i]> outliersSup:
            Venta.Precio[i] = Venta.Precio2[i] 
    Venta.drop(columns=['Precio2'], inplace=True)

    #Tratamiento de Outliers columna cantidad
    Q1 = Venta["Cantidad"].quantile(0.25)
    Q3 = Venta["Cantidad"].quantile(0.75)

    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR))
    mask = Venta["Cantidad"]<outliersSup
    ventaCantSinOut = Venta[mask]
    cantidadMedia = round(ventaCantSinOut.Cantidad.mean())
    Venta.loc[(Venta['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia #2
    Venta['Total Venta'] = Venta['Precio']*Venta['Cantidad']
    Venta.to_csv (r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Ventas_Normalizadas.csv', index = False, header=True)

In [36]:
CargaVentas()

C:\Users\maty_\AppData\Local\Temp\ipykernel_17348\3068816956.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Venta.Precio[i] = Venta.Precio2[i]


Tabla Compras

In [37]:
def CargaCompras():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Compr*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    Compra=pd.concat(df_list)
    #TABLA Compra
    Q1 = Compra["Precio"].quantile(0.25)
    Q3 = Compra["Precio"].quantile(0.75)
    #print(Q1)
    #print(Q3)
    #Calculo Rango interquartilico
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR)) 
    #print(outliersSup)

    #Uso de mascara para filtrar
    mask = Compra["Precio"]<outliersSup
    CompraSinOut2 = Compra[mask]
    #CompraSinOut2.describe()

    #Compruebo cantidad de outliers
    #Compra.shape[0]-CompraSinOut2.shape[0]

    #Outliers de precio
    Precio_Producto= CompraSinOut2.groupby(["IdProducto"])["Precio"].mean().reset_index()
    Precio_Producto= Precio_Producto.rename(columns= {"Precio":"Precio2"})

    Compra = pd.merge(Compra,Precio_Producto, how="left", on=["IdProducto"])
    Compra.sort_values("IdCompra")

    #Se reemplazan los 0 de precios2 por el valor de la media de los precios sin outliers
    Compra.Precio2.fillna(CompraSinOut2.Precio.mean(),inplace = True)

    #Se reemplazan con 0 y 1 todos los valores de precios vacios y cantidades
    Compra.Precio.fillna(0, inplace= True)
    Compra.Cantidad.fillna(1, inplace= True)

    #Hago columnas de unos
    Compra['Outliers'] = np.ones(Compra.shape[0])

    #Marco con 0 los outliers y despues los reemplazo con los valores de precio2
    Compra.loc[(Compra['Precio']>=outliersSup),'Outliers'] = 0

    #Reemplazo 0 y los outliers por el precio promedio sin outliers (precio2)
    for i in range(0,Compra.shape[0]):
        if Compra.Precio[i] == 0 or Compra.Precio[i]> outliersSup:
            Compra.Precio[i] = Compra.Precio2[i] #PRECIO2 ES EL PRECIO PROMEDIO SIN OUTLIERS
    Compra.drop(columns=['Precio2'], inplace=True)


    #Tratamiento de Outliers columna cantidad
    Q1 = Compra["Cantidad"].quantile(0.25)
    Q3 = Compra["Cantidad"].quantile(0.75)

    #Se calcula el rango intercuartilico IQR.
    IQR = Q3 -Q1
    outliersSup = (Q3 + (1.5*IQR))
    mask = Compra["Cantidad"]<outliersSup
    ventaCantSinOut = Compra[mask]
    cantidadMedia = round(ventaCantSinOut.Cantidad.mean())
    Compra.loc[(Compra['Cantidad'] >= outliersSup),"Cantidad"] = cantidadMedia 
    Compra['Total_Compra'] = Compra['Precio']*Compra['Cantidad']
    Compra.to_csv(r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Compras_Normalizadas.csv', index = False, header=True)


In [38]:
CargaCompras()

C:\Users\maty_\AppData\Local\Temp\ipykernel_17348\550835351.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Compra.Precio[i] = Compra.Precio2[i] #PRECIO2 ES EL PRECIO PROMEDIO SIN OUTLIERS


LOCALIDADES

In [39]:
def CargaLocalidades():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Localida*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "utf-8")
        df_list.append(df)
    global Localidad
    Localidad=pd.concat(df_list)
    #Tabla localidad
    #Localidad.head()
    #Localidad.isna().sum()
    Localidad['nombre']=Localidad.apply(lambda x: 'Ciudad de Buenos Aires' if x['municipio_nombre']=='Capital Federal' else x['nombre'], axis=1)

    #Completo los vacios de departamento_id y departamento_nombre
    #Localidad.isna().sum()
    Localidad["departamento_id"].fillna("Sin datos", inplace=True)
    Localidad["departamento_nombre"].fillna("Sin datos", inplace=True)

    #Reemplazo acentos en columna provincia y localidad

    Localidad["provincia_nombre"]=[x.replace('á',"a") for x in Localidad["provincia_nombre"]]
    Localidad["provincia_nombre"]=[x.replace('é',"e") for x in Localidad["provincia_nombre"]]
    Localidad["provincia_nombre"]=[x.replace('í',"i") for x in Localidad["provincia_nombre"]]
    Localidad["provincia_nombre"]=[x.replace('ó',"o") for x in Localidad["provincia_nombre"]]
    Localidad["provincia_nombre"]=[x.replace('ú',"u") for x in Localidad["provincia_nombre"]]

    Localidad["nombre"]=[x.replace('á',"a") for x in Localidad["nombre"]]
    Localidad["nombre"]=[x.replace('é',"e") for x in Localidad["nombre"]]
    Localidad["nombre"]=[x.replace('í',"i") for x in Localidad["nombre"]]
    Localidad["nombre"]=[x.replace('ó',"o") for x in Localidad["nombre"]]
    Localidad["nombre"]=[x.replace('ú',"u") for x in Localidad["nombre"]]


    #Renombro columnas que tienen valores de Latitud y Longitud
    Localidad= Localidad.rename(columns= {"centroide_lat":"Latitud"})
    Localidad= Localidad.rename(columns= {"centroide_lon":"Longitud"})


  
    Provincia['Provincia'] = Localidad.provincia_nombre
    Provincia['IdProvincia'] = Localidad.provincia_id

    #Saco duplicados
    Provincia.drop_duplicates(inplace=True)

    #Dropeo columnas que no sean necesarias
    Localidad.drop(columns='departamento_nombre', inplace=True)

    #Cambio nombres de columnas
    Localidad= Localidad.rename(columns= {"id":"IdLocalidad", "nombre":"Localidad","provincia_id":"IdProvincia", "departamento_id":"IdDepartamento", "provincia_nombre":"Provincia"})

    #Capitalizo columna nombre
    Localidad["Localidad"]= Localidad["Localidad"].str.title()

    #Dropeo localidades duplicadas en la misma provincia
    Localidad.drop_duplicates(subset=["Localidad","IdProvincia"], inplace=True)
    Localidad.reset_index(inplace=True)

    #Dropeo columnas que no sean necesarias
    Localidad.drop(['categoria','fuente', 'localidad_censal_id', 'localidad_censal_nombre', 'municipio_id','municipio_nombre','IdDepartamento' ], axis = 'columns', inplace=True)

    #Agrego una fila sin datos para llenar el merge despues y que no me queden vacios
    Localidad = Localidad.append({'Latitud':0,'Longitud':0, 'IdLocalidad':0, 'Localidad':'Sin Datos','Provincia':'Sin Datos'}, ignore_index = True)
    Localidad.to_csv(r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Localidades_Normalizadas.csv', index = False, header=True)
    Provincia.to_csv(r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Provincias_Normalizadas.csv', index = False, header=True)
    
    

In [40]:
CargaLocalidades()

C:\Users\maty_\AppData\Local\Temp\ipykernel_17348\1464881675.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Localidad = Localidad.append({'Latitud':0,'Longitud':0, 'IdLocalidad':0, 'Localidad':'Sin Datos','Provincia':'Sin Datos'}, ignore_index = True)


CARGA Y TRATAMIENTO DE CLIENTES

In [41]:
def CargaClientes():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Client*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ';',encoding = "utf-8", usecols=["ID","Provincia","Nombre_y_Apellido","Domicilio","Telefono","Edad","Localidad","X","Y","col10"])
        df_list.append(df)
    Cliente=pd.concat(df_list)
    
    #TABLA CLIENTE
    #Cliente.head()
    #Cliente.info()
    #Cliente.shape
    #Cliente.isnull().sum()

    #Drop col10
    Cliente.drop(columns= 'col10', inplace = True)


    #Cambio nombres de columnas
    Cliente= Cliente.rename(columns= {"ID":"IdCliente"})
    Cliente= Cliente.rename(columns= {"X":"Longitud"})
    Cliente= Cliente.rename(columns= {"Y":"Latitud"})

    #Letra capital
    Cliente["Nombre_y_Apellido"]= Cliente["Nombre_y_Apellido"].str.title()
    Cliente["Domicilio"]= Cliente["Domicilio"].str.title()
    Cliente["Localidad"]= Cliente["Localidad"].str.title()

    #Reemplazo de datos faltantes
    Cliente.Provincia.fillna("Sin Datos", inplace=True)
    Cliente.Nombre_y_Apellido.fillna("Sin Datos", inplace=True)
    Cliente.Localidad.fillna("Sin Datos", inplace=True)
    Cliente.Telefono.fillna("Sin Datos", inplace=True)
    Cliente.Domicilio.fillna("Sin Datos", inplace=True)

    #Reemplazo las comas  por puntos en latitud y longitud
    Cliente["Latitud"] = Cliente["Latitud"].str.replace(",", ".")
    Cliente["Longitud"] = Cliente["Longitud"].str.replace(",", ".")

    #Reemplazo nulos en lat y lon con 0
    Cliente.Latitud.fillna(0.0, inplace=True)
    Cliente.Longitud.fillna(0.0, inplace=True)

    #Normalizo provincia de Buenos Aires
    Cliente["Provincia"]=Cliente.apply(lambda x: "Buenos Aires" if x["Provincia"]=="Ciudad de Buenos Aires"  else x["Provincia"], axis=1)

    #Cambio de letras con acentos por letras sin acentos para que coincida fuzzy
    Cliente["Provincia"]=[x.replace('á',"a") for x in Cliente["Provincia"]]
    Cliente["Provincia"]=[x.replace('é',"e") for x in Cliente["Provincia"]]
    Cliente["Provincia"]=[x.replace('í',"i") for x in Cliente["Provincia"]]
    Cliente["Provincia"]=[x.replace('ó',"o") for x in Cliente["Provincia"]]
    Cliente["Provincia"]=[x.replace('ú',"u") for x in Cliente["Provincia"]]

    Cliente["Localidad"]=[x.replace('á',"a") for x in Cliente["Localidad"]]
    Cliente["Localidad"]=[x.replace('é',"e") for x in Cliente["Localidad"]]
    Cliente["Localidad"]=[x.replace('í',"i") for x in Cliente["Localidad"]]
    Cliente["Localidad"]=[x.replace('ó',"o") for x in Cliente["Localidad"]]
    Cliente["Localidad"]=[x.replace('ú',"u") for x in Cliente["Localidad"]]

    #Merge con Localidad y Cliente para traer IdLocalidad a Cliente
    Cliente = pd.merge(Cliente,Localidad, how="left", on=["Provincia","Localidad"])

    #Reemplazo Latitud y Longitud sin datos con los de tabla Localidad
    Cliente["Longitud_x"]=Cliente.apply(lambda x: x["Longitud_y"] if x["Longitud_x"]==0  else x["Longitud_x"], axis=1)
    Cliente["Latitud_x"]=Cliente.apply(lambda x: x["Latitud_y"] if x["Latitud_x"]==0  else x["Latitud_x"], axis=1)

    #Renombro columnas
    Cliente= Cliente.rename(columns= {"Longitud_x":"Longitud"})
    Cliente= Cliente.rename(columns= {"Latitud_x":"Latitud"})

    #Paso lat y long a float
    Cliente["Latitud"]= Cliente.Latitud.astype(float)
    Cliente["Longitud"]= Cliente.Longitud.astype(float)

    #Elimino columnas que se agregaron en el merge
    Cliente.drop(['index'], axis = 'columns', inplace=True)

    #Funcion que encuentra localidades trae idlocalidad e id provincia
    def verificarLocalidadCliente(row):
        ratiomayor=0
        for i in range(0,Localidad.shape[0]):
                Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), Localidad.Localidad[i].lower())
                if Ratio>ratiomayor:
                        ratiomayor=Ratio
                        miloc=Localidad.Localidad[i]
                        id=Localidad.IdLocalidad[i]
                        idprov=Localidad.IdProvincia[i]
        row["Localidad"]=miloc
        row["IdLocalidad"]=id
        row["IdProvincia"]=idprov
        return row

    #Llevo los clientes que no tienen IdLocalidad a un dataframe nuevo y aplico funcion
    Cliente_nulos= Cliente.loc[Cliente['IdLocalidad'].isnull() == True].copy()
    Cliente_nulos= Cliente_nulos.apply(verificarLocalidadCliente,axis=1)
    #Borro columnas innecesarias para el merge
    Cliente_nulos.drop(columns=["Latitud_y","Longitud_y","Domicilio","Telefono","Edad"],inplace=True)

    #Merge de Cliente con DF Cliente con IDlocalidad Nulos (ya corregido)
    Cliente = pd.merge(Cliente,Cliente_nulos, how="left", on=["IdCliente"])

    #Completo IdLocalidad en tabla Cliente original con 0 para prepararlos para el reemplazo
    Cliente["IdLocalidad_x"]= Cliente["IdLocalidad_x"].fillna(0)
    Cliente["IdProvincia_x"]= Cliente["IdProvincia_x"].fillna(0)

    #Realizo reemplazo de IdLocalidad y IdProvincia traidos con el merge
    Cliente["IdLocalidad_x"]=Cliente.apply(lambda x: x["IdLocalidad_y"] if x["IdLocalidad_x"]==0  else x["IdLocalidad_x"], axis=1)
    Cliente["IdProvincia_x"]=Cliente.apply(lambda x: x["IdProvincia_y"] if x["IdProvincia_x"]==0  else x["IdProvincia_x"], axis=1)

    #25 Localidad sin IdLocalidad - se lo completa como sin dato
    Cliente["IdLocalidad_x"]=Cliente["IdLocalidad_x"].fillna(0)
    #Elimino columnas innecesarias
    Cliente.drop(['Latitud_y','Longitud_y','Provincia_y','Nombre_y_Apellido_y','Localidad_y', 'Longitud_y', 'IdLocalidad_y','IdProvincia_y'], axis = 'columns', inplace=True)
    #Renombro columnas
    Cliente= Cliente.rename(columns= {"Provincia_x":"Provincia"})
    Cliente= Cliente.rename(columns= {"Nombre_y_Apellido_x":"Nombre_y_Apellido"})
    Cliente= Cliente.rename(columns= {"Localidad_x":"Localidad"})
    Cliente= Cliente.rename(columns= {"Longitud_x":"Longitud"})
    Cliente= Cliente.rename(columns= {"Latitud_x":"Latitud"})
    Cliente= Cliente.rename(columns= {"IdLocalidad_x":"IdLocalidad"})
    Cliente= Cliente.rename(columns= {"IdProvincia_x":"IdProvincia"})
    Cliente.to_csv(r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Clientes_Normalizados.csv', index = False, header=True)

In [42]:
CargaClientes()

PROVEEDORES CARGA Y TRATAMIENTO

In [43]:
def CargaProveedores():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Proveedo*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ',',encoding = "ansi")
        df_list.append(df)
    Proveedor=pd.concat(df_list)
    #Tabla proveedores
    #Dropeo departamen
    #Proveedor.drop(columns= 'departamen', inplace = True)

    #Renombro columnas
    Proveedor= Proveedor.rename(columns= {"IDProveedor":"IdProveedor", "Nombre":"Proveedor","Address":"Domicilio", "City":"Localidad", "State":"Provincia", "Country":"Pais", "departamen":"Departamento"})

    #Reemplazo acentos en Provincia y Localidad
    Proveedor["Provincia"]=[x.replace('á',"a") for x in Proveedor["Provincia"]]
    Proveedor["Provincia"]=[x.replace('é',"e") for x in Proveedor["Provincia"]]
    Proveedor["Provincia"]=[x.replace('í',"i") for x in Proveedor["Provincia"]]
    Proveedor["Provincia"]=[x.replace('ó',"o") for x in Proveedor["Provincia"]]
    Proveedor["Provincia"]=[x.replace('ú',"u") for x in Proveedor["Provincia"]]

    Proveedor["Localidad"]=[x.replace('á',"a") for x in Proveedor["Localidad"]]
    Proveedor["Localidad"]=[x.replace('é',"e") for x in Proveedor["Localidad"]]
    Proveedor["Localidad"]=[x.replace('í',"i") for x in Proveedor["Localidad"]]
    Proveedor["Localidad"]=[x.replace('ó',"o") for x in Proveedor["Localidad"]]
    Proveedor["Localidad"]=[x.replace('ú',"u") for x in Proveedor["Localidad"]]

    #Capitalizo datos
    Proveedor['Domicilio'] = Proveedor['Domicilio'].str.title()
    Proveedor['Localidad'] = Proveedor['Localidad'].str.title()
    Proveedor['Provincia'] = Proveedor['Provincia'].str.title()
    Proveedor['Pais'] = Proveedor['Pais'].str.title()
    Proveedor['Departamento'] = Proveedor['Departamento'].str.title()

    #Lleno vacios
    #Proveedor.isna().sum()
    Proveedor.Proveedor.fillna('Sin Datos', inplace=True)

    Provincia.reset_index(inplace=True)

    #NORMALIZACION PROVINCIA
    lista_provincia=[]
    for i in range(Provincia.shape[0]):
        lista_provincia.append(Provincia.loc[i,"Provincia"])


    #Funcion para encontrar parecios con fuzz
    def verificarProvinciaProveedor(row):
        if(row["Provincia"] not in lista_provincia):
                ratiomayor=0
                for i in range(0,Provincia.shape[0]):
                    Ratio= fuzz.token_sort_ratio(row["Provincia"].lower(), Provincia.Provincia[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=Provincia.Provincia[i]
                row["Provincia"]=miloc
        return row

    #Corrijo CABA
    Proveedor['Provincia']=Proveedor.apply(lambda x: 'Buenos Aires' if x['Provincia']=='CABA' else x['Provincia'],axis=1)

    #Aplico funcion
    Proveedor=Proveedor.apply(verificarProvinciaProveedor, axis=1)

    #Utilizo esta funcion para cambiar Ciudad de Buenos Aires que con la otra no se puede cambiar, luego utiliza la misma funcion que arriba para las demas localidades
    ver_loc = list(Proveedor['Localidad'].unique())

    #for n in ver_loc:
    #   print(n)


    # Creamos una lista con las denominaciones más comunes que puede recibir la ciudad de buenos aires
    list_BSAS = ["Ciudad de Buenos Aires", "Capital Federal", "Cap Fed", "CABA"]
    nom_normalizado = [ ]
    loc1 = " "
    loc2 = " "
    for m in list_BSAS:
        loc1 = m
        for n in ver_loc:
            loc2 = n
            Ratio = fuzz.ratio(loc1.lower(),loc2.lower())
            Partial_Ratio = fuzz.partial_ratio(loc1.lower(),loc2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(loc1, loc2)
            Token_Set_Ratio = fuzz.token_set_ratio(loc1, loc2)
            if ((Partial_Ratio > 70) and (Token_Set_Ratio > 70)):
                nom_normalizado.append(loc2)
    print(nom_normalizado)

    for n in range (len(nom_normalizado)):
        Proveedor['Localidad'].loc[(Proveedor['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'


    #Normalizacion Localidad
    lista_localidades = []
    for i in range(Localidad.shape[0]):
        lista_localidades.append(Localidad.loc[i,'Localidad'])
        

    def verificarLocalidad(row):
        if(row["Localidad"] not in lista_localidades):
                ratiomayor=0
                for i in range(0,Localidad.shape[0]):
                    Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), Localidad.Localidad[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=Localidad.Localidad[i]
                row["Localidad"]=miloc
        return row

    #Aplico la funcion
    Proveedor = Proveedor.apply(verificarLocalidad, axis=1)

    #Hago merge para traer el idlocalidad
    Proveedor = pd.merge(Proveedor,Localidad, how="left", on=["Localidad","Provincia"])
    Proveedor.drop(['Pais','Departamento','index','Latitud','Longitud','Localidad' ], axis = 'columns', inplace=True)
    Proveedor.to_csv (r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Proveedores_Normalizados.csv', index = False, header=True)

In [44]:
CargaProveedores()

[]


In [45]:
def CargaSucursal():
    #ds_path= pathlib.Path("Datasets")
    #print(ds_path)
    df_list=[]
    #archivo=glob.glob('.csv')
    #print("Hola")
    for file in glob.glob('Sucursa*.csv'):
        #print(file)
        df=pd.read_csv(file,delimiter = ';',encoding = "utf-8")
        df_list.append(df)
    Sucursal=pd.concat(df_list) 
    #TABLA Sucursal
    #Sucursal.isna().sum()
    #Renombro columna ID
    Sucursal= Sucursal.rename(columns= {"ID":"IdSucursal"})

    #Reemplazo letras con acento en provincia y localidad
    Sucursal["Provincia"]=[x.replace('á',"a") for x in Sucursal["Provincia"]]
    Sucursal["Provincia"]=[x.replace('é',"e") for x in Sucursal["Provincia"]]
    Sucursal["Provincia"]=[x.replace('í',"i") for x in Sucursal["Provincia"]]
    Sucursal["Provincia"]=[x.replace('ó',"o") for x in Sucursal["Provincia"]]
    Sucursal["Provincia"]=[x.replace('ú',"u") for x in Sucursal["Provincia"]]

    Sucursal["Localidad"]=[x.replace('á',"a") for x in Sucursal["Localidad"]]
    Sucursal["Localidad"]=[x.replace('é',"e") for x in Sucursal["Localidad"]]
    Sucursal["Localidad"]=[x.replace('í',"i") for x in Sucursal["Localidad"]]
    Sucursal["Localidad"]=[x.replace('ó',"o") for x in Sucursal["Localidad"]]
    Sucursal["Localidad"]=[x.replace('ú',"u") for x in Sucursal["Localidad"]]


    # Modifico tipo de dato de latitud y longitud
    Sucursal["Longitud"] = Sucursal["Longitud"].str.replace(",", ".").astype(float)
    Sucursal["Latitud"] = Sucursal["Latitud"].str.replace(",", ".").astype(float)

    #Normalizacion de provincia
    lista_provincia=[]
    for i in range(Provincia.shape[0]):
        lista_provincia.append(Provincia.loc[i,"Provincia"])

    #Funcion para encontrar parecios con fuzz
    def verificarProvincia(row):
        if(row["Provincia"] not in lista_provincia):
                ratiomayor=0
                for i in range(0,Provincia.shape[0]):
                    Ratio= fuzz.token_sort_ratio(row["Provincia"].lower(), Provincia.Provincia[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miloc=Provincia.Provincia[i]
                row["Provincia"]=miloc
        return row

    
    #Corrijo CABA
    Sucursal['Provincia']=Sucursal.apply(lambda x: 'Buenos Aires' if x['Provincia']=='CABA' else x['Provincia'],axis=1)

    #Aplico funcion
    Sucursal=Sucursal.apply(verificarProvincia, axis=1)

    #Normalizo las variaciones de ciudad de buenos aires
    ver_loc = list(Sucursal['Localidad'].unique())

    #Chequeo ver_loc
    #for n in ver_loc:
    #   print(n)


    # Denominaciones más comunes que puede recibir la ciudad de buenos aires y las guardo en una lista
    list_BSAS = ["Ciudad de Buenos Aires", "Capital Federal", "Cap Fed", "CABA"]
    nom_normalizado = [ ]
    loc1 = " "
    loc2 = " "
    for m in list_BSAS:
        loc1 = m
        for n in ver_loc:
            loc2 = n
            Ratio = fuzz.ratio(loc1.lower(),loc2.lower())
            Partial_Ratio = fuzz.partial_ratio(loc1.lower(),loc2.lower())
            Token_Sort_Ratio = fuzz.token_sort_ratio(loc1, loc2)
            Token_Set_Ratio = fuzz.token_set_ratio(loc1, loc2)
            if ((Partial_Ratio > 70) and (Token_Set_Ratio > 70)):
                nom_normalizado.append(loc2)
    print(nom_normalizado)

    for n in range (len(nom_normalizado)):
        Sucursal['Localidad'].loc[(Sucursal['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'


    #Normalizacion Localidad
    lista_localidades = []
    for i in range(Localidad.shape[0]):
        lista_localidades.append(Localidad.loc[i,'Localidad'])

    def verificarLocalidad(row):
        if(row["Localidad"] not in lista_localidades):
                ratiomayor=0
                for i in range(0,Localidad.shape[0]):
                    Ratio= fuzz.token_sort_ratio(row["Localidad"].lower(), Localidad.Localidad[i].lower())
                    if Ratio>ratiomayor:
                            ratiomayor=Ratio
                            miprov=Localidad.Localidad[i]
                row["Localidad"]=miprov
        return row

    Sucursal = Sucursal.apply(verificarLocalidad, axis=1)

    #MERGE PARA TRAER ID Localidad
    Sucursal = pd.merge(Sucursal,Localidad, how="left", on=["Localidad","Provincia"])
    Sucursal.drop(['Latitud_y','Longitud_y', 'Localidad', 'Provincia', 'index', 'Latitud_x', 'Longitud_x' ], axis = 'columns', inplace=True)
    Sucursal.to_csv(r'C:\Users\maty_\Desktop\Henry DS\LABS\DS-PI-ProyectoIndividual\Datasetnuevos\Sucursales_Normalizadas.csv', index = False, header=True)

In [46]:
CargaSucursal()

['Ciudad de Buenos Aires', 'Cdad de Buenos Aires', 'Capital', 'Capital Federal', 'Cap.   Federal', 'CapFed', 'Cap. Fed.', 'CABA']


C:\Users\maty_\AppData\Local\Temp\ipykernel_17348\1881282846.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sucursal['Localidad'].loc[(Sucursal['Localidad']==str(nom_normalizado[n]))]='Ciudad de Buenos Aires'
